<a href="https://colab.research.google.com/github/JZLGoodLuck/JZLGoodLuck.github.io/blob/main/notebooks/HRN_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Enable GPUs
First, you'll need to enable GPUs for the notebook:

*   Navigate to Edit→Notebook Settings
*   select GPU from the Hardware Accelerator drop-down

# Step 1: Installation
It takes about 30 minutes.

In [2]:
!pip install torch torchvision torchaudio
!pip install tensorflow
!pip install "modelscope[cv]" -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
!pip install git+https://github.com/NVlabs/nvdiffrast.git
!pip install git+https://github.com/facebookresearch/pytorch3d.git

Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
  Cloning https://github.com/NVlabs/nvdiffrast.git to /tmp/pip-req-build-3au559ez
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast.git /tmp/pip-req-build-3au559ez
  Resolved https://github.com/NVlabs/nvdiffrast.git to commit 729261dc64c4241ea36efda84fbf532cc8b425b8
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-qu44y24p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-qu44y24p
  Resolved https://github.com/facebookresearch/pytorch3d.git to commit 055ab3a2e3e611dff66fa82f632e62a315f3b5e7
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch3d: filename=pytorch3d-0.7.8-cp310-cp310-linux_x86_64.whl size=59604557 sha256=240246e030e3a210f66acd1fbfe92d7ad3e6beab2017992fc1af03eaa322d379
  Stored in direct

# Step 2: Model initialization
Initialize the pipeline and download the pretrained weights from modelscope.

In [3]:
import os
import cv2
from moviepy.editor import ImageSequenceClip
from modelscope.models.cv.face_reconstruction.utils import write_obj
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

face_reconstruction = pipeline(Tasks.face_reconstruction, model='damo/cv_resnet50_face-reconstruction', model_revision='v2.0.0-HRN')

  if event.key is 'enter':



2024-11-29 15:55:05,086 - modelscope - INFO - Use user-specified model revision: v2.0.0-HRN


  check_for_updates()

2024-11-29 15:56:41,489 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/damo/cv_resnet50_face-reconstruction
2024-11-29 15:56:41,492 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/damo/cv_resnet50_face-reconstruction.
2024-11-29 15:56:41,496 - modelscope - INFO - initialize model from /root/.cache/modelscope/hub/damo/cv_resnet50_face-reconstruction


initialize network with normal
initialize network with normal


2024-11-29 15:56:45,601 - modelscope - WARNING - No preprocessor field found in cfg.
2024-11-29 15:56:45,603 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-11-29 15:56:45,608 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/root/.cache/modelscope/hub/damo/cv_resnet50_face-reconstruction'}. trying to build by task and model information.
2024-11-29 15:56:45,608 - modelscope - WARNING - No preprocessor key ('face_reconstruction', 'face-reconstruction') found in PREPROCESSOR_MAP, skip building preprocessor.


loading the model from /root/.cache/modelscope/hub/damo/cv_resnet50_face-reconstruction/pytorch_model.pt


Instructions for updating:
Use tf.gfile.GFile.


# Step 3: Run

In [6]:
def save_results(result, save_root):
    os.makedirs(save_root, exist_ok=True)

    # export obj and texture
    mesh = result[OutputKeys.OUTPUT]['mesh']
    texture_map = result[OutputKeys.OUTPUT_IMG]
    mesh['texture_map'] = texture_map
    write_obj(os.path.join(save_root, 'hrn_mesh_mid.obj'), mesh)

    # export rotation video
    frame_list = result[OutputKeys.OUTPUT]['frame_list']
    video = ImageSequenceClip(sequence=frame_list, fps=30)
    video.write_videofile(
        os.path.join(save_root, 'rotate.mp4'), fps=30, audio=False)
    del frame_list

    # save visualization image
    vis_image = result[OutputKeys.OUTPUT]['vis_image']
    cv2.imwrite(os.path.join(save_root, 'vis_image.jpg'), vis_image)

    print(f'Output written to {os.path.abspath(save_root)}')

result = face_reconstruction('https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/face_reconstruction.jpg')
save_results(result, './face_reconstruction_results')

Moviepy - Building video ./face_reconstruction_results/rotate.mp4.
Moviepy - Writing video ./face_reconstruction_results/rotate.mp4



Moviepy - Done !
Moviepy - video ready ./face_reconstruction_results/rotate.mp4
Output written to /content/face_reconstruction_results


# Step 4: Visualize results

In [5]:
from matplotlib import pyplot as plt
def vis_img(path):
  img = cv2.imread(path,-1)
  plt.imshow(img[..., ::-1])
  plt.axis("off")
  plt.show()

vis_img('./face_reconstruction_results/vis_image.jpg')

TypeError: 'NoneType' object is not subscriptable